In [1]:
#collecting data
import numpy as np
import scipy.io
mat = scipy.io.loadmat('dataset_connected_NYC.mat') #data10while tryin in terminal, start path with Documents

In [2]:
#friendship_new = mat['friendship_new']
friendship_new = mat['friendship_new']
friendship_old = mat['friendship_old']#new_friendship_old
selected_checkins = mat['selected_checkins'] #new_checkins
selected_users_IDs = mat['selected_users_IDs'] #new_users_IDs

#print(min(map(min, friendship_old)))

print(np.unique(selected_users_IDs).shape)
print(selected_users_IDs.shape)
print(np.unique(selected_checkins[:,0]).shape)
print(np.unique(selected_checkins[:,2]).shape)
print(selected_checkins[:,2].shape)

(4024,)
(4024, 1)
(3785,)
(3628,)
(105961,)


In [3]:
qty = 10 #number of entities
num_ckn_wanted = 3 #number of checkin per user from the data


num_usr_chkn = np.zeros((len(selected_users_IDs),2)).astype(np.int64)
print(num_usr_chkn.shape)
for ele in selected_checkins[:,0]:
    num_usr_chkn[ele-1][0]+=1
    num_usr_chkn[ele-1][1]=ele
num_usr_chkn=num_usr_chkn[np.argsort(num_usr_chkn[:,0])]
#taking 50 IDs with num_ckn_wanted checkins
k=qty
new_users_IDs=[]
for i in num_usr_chkn:
    if i[0]==num_ckn_wanted and k>0:
        new_users_IDs.append(i[1])
        k-=1
print(new_users_IDs)
temp=[]
for i in range(len(new_users_IDs)):
    k = np.where(selected_checkins[:,0]==new_users_IDs[i]) 
    #check that the value is less than size of checkins and number of values is = num_ckn_wanted
    for j in k:
        temp.append(selected_checkins[j,:])
selected_checkins = np.array(temp)[:,0,:]
print(len(selected_checkins))

(4024, 2)
[2821, 1958, 39, 40, 1777, 1504, 3235, 1497, 3191, 3129]
10


In [4]:
train_size = int((80/100)*np.size(selected_checkins,0))
test_size = np.size(selected_checkins,0) - train_size
train_checkins = selected_checkins[:train_size]
test_checkins = selected_checkins[train_size:]
orig_checkins = selected_checkins
selected_checkins = train_checkins
print(len(selected_checkins))

8


In [5]:
k=np.sort(selected_checkins[:,2])
print(k)
for i in range(len(k)):
    t=np.array(np.where(selected_checkins[:,2]==k[i]))
    for j in range(len(t[0])):
        selected_checkins[t[0][j],2]=i+j+1
print(selected_checkins[:,2])

[ 3968  5333  7415  7607  8862  9336 19754 75235]
[5 3 8 6 7 4 1 2]


In [6]:
chk = np.sort(selected_checkins[:,2])
print(chk)

[1 2 3 4 5 6 7 8]


In [7]:
print(selected_checkins)

[[2821  108    5   67]
 [1958  109    3  307]
 [  39   65    8  337]
 [  40   27    6  113]
 [1777   88    7   47]
 [1504   82    4  283]
 [3235  144    1  300]
 [1497   22    2   44]]


In [8]:
#swapping the col locations
selected_checkins[:,[0,2]]=selected_checkins[:,[2,0]]
#the configuration of selected_checkin now
#location, hours in the day, users_ID, location category

# 1. rebuild node index
offset1 = max(selected_checkins[:,0]);
dumy, dumy, n = np.unique(selected_checkins[:,1],return_index=True,return_inverse=True, axis=0)


selected_checkins[:,1] = n+offset1+1; #n is the indices of the unique values in selected_checkins[:,1]
offset2 = max(selected_checkins[:,1]);
dumy, dumy, n = np.unique(selected_checkins[:,2],return_index=True,return_inverse=True, axis=0)

selected_checkins[:,2] = n+offset2+1;
offset3 = max(selected_checkins[:,2]);
dumy, dumy, n = np.unique(selected_checkins[:,3],return_index=True,return_inverse=True, axis=0)
selected_checkins[:,3] = n+offset3+1;

num_node_total = max(map(max, selected_checkins)) #max of the entire matrix = 8117

In [9]:
print(selected_checkins.shape)
print(min(selected_checkins[:,0]))
print(max(selected_checkins[:,0]))
print(min(selected_checkins[:,1]))
print(max(selected_checkins[:,1]))
print(min(selected_checkins[:,2]))
print(max(selected_checkins[:,2]))
print(min(selected_checkins[:,3]))
print(max(selected_checkins[:,3]))

(8, 4)
1
8
9
16
17
24
25
32


In [10]:
import random
unique_checkins = np.unique(selected_checkins[:,0])
selected_location = random.sample(list(unique_checkins),len(unique_checkins))
#random.sample randomly selectes 2nd arg number of elements from 1st arg
#selecting all locations so that no loss of information occurs
selected_location = np.array(selected_location) #for function compatibility
print(len(selected_location))
print(len(selected_checkins[:,0]))

8
8


In [11]:
#uncomment for debugging
#print(selected_location)
#print(selected_checkins[:,2])

In [12]:
# 2. prepare checkins per location (fast)
#user_chechkins is a cell in MATLAB; using Python List for the same
    user_checkins = [[] for _ in range(selected_location.shape[0])]
    user_checkins_counter=np.zeros((len(selected_location),1)).astype(np.int64)
    ind = selected_checkins[:,0].argsort(axis=0)
    temp_checkins = np.array([selected_checkins[i,:] for i in ind]) #will not exactly be similar to MATLAB op
    u,m,n = np.unique(temp_checkins[:,0],return_index=True,return_inverse=True, axis=0)
    m=m.reshape(-1,1)
    u=np.array(u)
    u= u-min(u)
    counters = np.vstack((m[1:],temp_checkins.shape[0]))-m #+1 is not there because indexing starts from 0
    ini_val=0 #min(selected_location)
    for i in range(0,u.shape[0]):
        user_checkins[u[i]-1].append(temp_checkins[ini_val:ini_val+counters[i,0],:])
        #print(i,"here1")
        user_checkins[u[i]-1]=np.array(user_checkins[u[i]-1])[0,:,:].T
        #print(i,"here2")
        user_checkins[u[i]-1].astype(np.int64)
        ini_val=ini_val+counters[i,0]
        user_checkins_counter[u[i]-1]=counters[i].astype(np.int64)
        #print(i,"here3")


In [13]:
print(len(user_checkins))

8


In [14]:
#print("user_checkins",user_checkins)
print("user_checkins_counter",user_checkins_counter)

user_checkins_counter [[1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]]


In [15]:
#trial for making walks

#selected_location = list(selected_location)
#print(selected_location)
#k = selected_location.pop(0)
#print(selected_location)
#selected_location.insert(0,k)
#print(selected_location)

In [16]:
# let's have a walk over all th locations
import random

selected_location = list(selected_location)
num_node = len(selected_location)
num_walk = 10
len_walk = num_node  #has to be smaller than len(selected_location)
print("selected_location length",len(selected_location))
walks = np.zeros((num_walk*num_node,len_walk),dtype = np.int64);
for ww in range(num_walk):
    for ii in range(num_node):
        seq = np.zeros((len_walk,),dtype=int)
        seq[0] = selected_location[ii]
        ins_later = selected_location.pop(ii)
        seq[1:]= np.random.choice(selected_location,len_walk-1,replace=False)  #selected nodes from selected_location without repetition
        selected_location.insert(ii,ins_later)
        walks[ii+(ww-1)*num_node,:] = seq
        
#preprocessing for removing 0 quantities
for i in range(walks.shape[0]):
    for j in range(walks.shape[1]):
        walks[i,j] = walks[i,j] +1
#print(min(map(min, walks))) #should be 1 and 4024


#saving these numbers in walk are creating out of index problem for embedding.py 
walks = walks - min(map(min, walks))

print(walks)

selected_location length 8
[[2 3 5 7 4 6 0 1]
 [7 1 2 6 0 5 4 3]
 [1 3 7 2 5 0 6 4]
 [6 1 0 2 5 7 4 3]
 [4 6 7 5 0 3 1 2]
 [5 0 3 6 7 1 4 2]
 [3 0 1 6 5 2 4 7]
 [0 3 6 7 4 5 2 1]
 [2 5 3 1 6 4 7 0]
 [7 5 0 1 2 6 3 4]
 [1 7 0 4 5 6 3 2]
 [6 4 0 7 1 2 5 3]
 [4 1 7 2 5 3 6 0]
 [5 0 2 1 7 6 3 4]
 [3 1 5 0 2 6 7 4]
 [0 6 2 4 1 5 7 3]
 [2 0 7 4 6 3 5 1]
 [7 4 2 6 5 0 1 3]
 [1 5 4 6 2 7 0 3]
 [6 2 1 5 3 0 4 7]
 [4 6 0 2 1 3 5 7]
 [5 4 6 7 2 1 3 0]
 [3 0 2 6 1 7 5 4]
 [0 4 3 5 6 7 2 1]
 [2 0 4 7 1 3 6 5]
 [7 4 5 6 0 2 3 1]
 [1 7 6 3 2 4 0 5]
 [6 0 2 7 3 4 1 5]
 [4 0 1 7 5 2 6 3]
 [5 1 3 4 2 6 7 0]
 [3 6 7 2 0 4 1 5]
 [0 4 1 3 6 7 5 2]
 [2 4 1 0 3 5 7 6]
 [7 6 3 0 4 5 2 1]
 [1 7 0 6 3 5 2 4]
 [6 3 5 0 2 4 1 7]
 [4 3 7 5 0 1 2 6]
 [5 7 3 4 6 0 1 2]
 [3 5 0 4 1 7 2 6]
 [0 4 3 2 1 6 7 5]
 [2 3 0 5 4 7 1 6]
 [7 3 6 1 0 2 4 5]
 [1 3 0 6 2 7 4 5]
 [6 0 5 3 2 1 4 7]
 [4 0 3 5 2 7 1 6]
 [5 4 6 0 7 3 1 2]
 [3 4 7 0 6 5 1 2]
 [0 2 5 7 1 4 3 6]
 [2 5 7 4 3 0 6 1]
 [7 6 0 2 4 3 5 1]
 [1 4 6 3 0 5 7 2]
 [6 

In [17]:
# 4. prepare negative sample table in advance (fast)
# location relationship
from collections import Counter

temptab = Counter(selected_checkins[:,0]) #len(temptab) = size(tab_degree,1) #Counter gives unique elements in r and their freq
print("temptab ",temptab,len(temptab))
tab2= [[] for _ in range(len(temptab))]

ind=0
for i in temptab:
    #print(i,temptab[i])
    tab2[ind].append(temptab[i])
tab2=np.array(tab2[:][0]) #when printing temptab, not in asc. order but here it becomes asc. wrt keys

tab1=np.unique(selected_checkins[:,0])
tab3=[[] for _ in range(len(tab1))]
tot=sum(tab2)
for i in range(len(tab1)):
    tab3[i]=round((tab2[i]*100)/tot,4)
tab_degree=np.stack((tab1,tab2,tab3),axis=1) # equivalent of tab_degree = tabulate(r) in MATLAB

freq = np.array([round(np.power(i,0.75),4) for i in tab_degree[:,2]] )

den=float(sum(freq))

neg_sam_table_social = np.repeat((1+tab_degree[:,0]),np.around(1000000*freq/sum(freq)).astype(np.int64))
neg_sam_table_social=neg_sam_table_social.astype(np.int64) # unigram with 0.75 power
#neg_sam_table_social[740] check for debugging
del temptab,tab1,tab2,tab3,tab_degree,freq,den


temptab  Counter({5: 1, 3: 1, 8: 1, 6: 1, 7: 1, 4: 1, 1: 1, 2: 1}) 8


In [18]:
#uncomment for debugging
#trii = Counter(neg_sam_table_social)
#print(trii)

In [19]:
#selected_checkins (4 columns): user_index, time (hour in a week), venue_index, venue_category_index

neg_sam_table_mobility_norm = [[] for _ in range(4)]
for ii in range(len(neg_sam_table_mobility_norm)):
    tab1=np.array([elem for elem in range(max(temp_checkins[:,ii]))])#tab1 has all elements;even with frequency 0
    #temp_checkins is basically array version of selected_checkins
    temptab = Counter(temp_checkins[:,ii]) #correct
    tab2= np.zeros((len(tab1),))
    for i in tab1:
        tab2[i]= temptab[i+1] #because in python, index starts from 0
    tab2 = np.array(tab2)
    tot=np.sum(tab2)
    tab3=np.zeros((len(tab1),))
    for i in range(len(tab1)):
        tab3[i]= np.round((tab2[i]*100)/tot,4) if (tot) else 0
    tab3=np.array(tab3)
    tab_degree=np.stack((tab1,tab2,tab3),axis=1)
    freq = np.array([np.round(np.power(i,0.75),4) for i in tab_degree[:,2]])
    den=float(sum(freq))
    ingoes = np.repeat((tab_degree[:,0]+1),np.around(100000*freq/sum(freq)).astype(np.int64))
    neg_sam_table_mobility_norm[ii].append(ingoes)

    del tab1,tab2,i,tab3,tab_degree,freq,ingoes,tot

neg_sam_table_mobility_norm = [np.array(x).astype(np.int64) for x in neg_sam_table_mobility_norm]

#neg_sam_table_mobility1 = neg_sam_table_mobility_norm[3][0]
#neg_sam_table_mobility1[5]
print(neg_sam_table_mobility_norm[0][0])
print(neg_sam_table_mobility_norm[1][0])
print(neg_sam_table_mobility_norm[2][0])
print(neg_sam_table_mobility_norm[3][0])

[1 1 1 ... 8 8 8]
[ 9  9  9 ... 16 16 16]
[17 17 17 ... 24 24 24]
[25 25 25 ... 32 32 32]


In [20]:
# LBSN2vec
dim_emb = 2 #128
num_epoch = 1
num_threads =  4
K_neg = 3 #10
win_size = 2 #10
learning_rate = 0.001

embs_ini = (np.random.uniform(size=(num_node_total,dim_emb))-0.5)/dim_emb 
temp = np.sum(np.power(embs_ini,2),axis=1)
embs_len = np.power(temp,0.5)
den = embs_len
for i in range(dim_emb-1):
    den=np.vstack((den,embs_len))
den=den.T
#print(den.shape)
#print(embs_ini.shape)
embs_ini = np.divide(embs_ini,den)

mobility_ratio = 0.2

In [21]:
import learn_LBSN2Vec_embedding

embs = learn_LBSN2Vec_embedding.driver_fn(walks.T,user_checkins, user_checkins_counter,embs_ini.T,learning_rate, 
                                K_neg,neg_sam_table_social, win_size, neg_sam_table_mobility_norm,
                                num_epoch, mobility_ratio);

this is num_w 80
this is num_wl 8
This is epoch  0


In [22]:
print(embs.shape)
print(np.min(temp_checkins))
print(np.max(temp_checkins))
print(temp_checkins.shape)
print("this is range of user "+ str(min(temp_checkins[:,0])))
print("this is range of user "+ str(max(temp_checkins[:,0])))
print("this is range of time "+ str(min(temp_checkins[:,1])))
print("this is range of time "+ str(max(temp_checkins[:,1])))
print("this is range of venue_index "+ str(min(temp_checkins[:,2])))
print("this is range of venue_index "+ str(max(temp_checkins[:,2])))
print("this is range of venue_category_index "+ str(min(temp_checkins[:,3])))
print("this is range of venue_category_index "+ str(max(temp_checkins[:,3])))

(2, 32)
1
32
(8, 4)
this is range of user 1
this is range of user 8
this is range of time 9
this is range of time 16
this is range of venue_index 17
this is range of venue_index 24
this is range of venue_category_index 25
this is range of venue_category_index 32


In [23]:
#normalizing

twodemb = np.zeros((embs.shape[1],2))
for i in range(twodemb.shape[0]):
    twodemb[i,0] = embs[0,i]
    twodemb[i,1] = embs[1,i]

print(twodemb)

[[-0.94214595  0.3352052 ]
 [ 0.62868589 -0.7776605 ]
 [-0.99091955 -0.13447322]
 [-0.88244842  0.47041647]
 [ 0.4221045   0.90654966]
 [ 0.29087344  0.95676307]
 [ 0.12909715  0.99163368]
 [-0.56441505  0.8254927 ]
 [ 0.64552621  0.76373942]
 [-0.86800778  0.49655059]
 [-0.30116521 -0.95357201]
 [-0.98869561  0.14994609]
 [ 0.25913871 -0.96584011]
 [ 0.91822673 -0.39605628]
 [ 0.14241592  0.98980698]
 [-0.53002075 -0.84798468]
 [ 0.571388   -0.82068082]
 [-0.59751385 -0.80185859]
 [-0.56986126 -0.82174092]
 [-0.88861339  0.45865809]
 [-0.94637253  0.32307893]
 [ 0.55865717 -0.82939942]
 [-0.58492623 -0.81108665]
 [ 0.12412202  0.99226696]
 [-0.53841734 -0.84267845]
 [-0.51419424 -0.85767387]
 [-0.07160606 -0.99743299]
 [-0.98032884 -0.19737253]
 [-0.837948   -0.54575026]
 [-0.23963498 -0.97086307]
 [-0.27178587 -0.96235779]
 [-0.57367882  0.81908067]]


In [24]:
#location prediction on the learnt embeddings
from scipy.spatial import distance
import time

count = 0

start_time = time.time()
for d in range(selected_checkins.shape[0]):
    test = selected_checkins[d]
    user = test[0]
    hour = test[1]
    dist = []
    
    for i in range(min(selected_checkins[:,2]),max(selected_checkins[:,2])+1):
        summ = (1 - distance.cosine(twodemb[user-1,:],twodemb[i-1,:]))+(1 - distance.cosine(twodemb[hour-1,:],twodemb[i-1,:]))
        dist.append((abs(summ),i))
        
    dist = sorted(dist, key=lambda student: student[0])
    if test[3] in (np.array(dist[0:10])[:,1]):
        count+=1
print("this is count "+str(count))
print("this is count fraction "+str(count/selected_checkins.shape[0]))
print("time taken is ",(time.time() - start_time))


this is count 0
this is count fraction 0.0
time taken is  0.04414224624633789
